# RAG Pipepline from Scratch 

RAG (Retrieval Augmented Generation) has to goal to take information and pass it to a Large Language Model (LLM) so it can generate outputs based on that information. 

* **Retrieval**: Find relevant information given a user query. I.e. What are the macronutrients and what do they do -> Retrives any passages of text related to the macronutrients from a nutritien textbook. 
* **Augmented**: We want to take the relevant information from our data and then augment our imput (prompt) to an LLM with that relevant information. 
* **Generation**: Take the first two stepes and pass them to an LLM for a good output. 

Why RAG? 
The main goal of RAG is to improve the generation output of LLMs.
1. Prevent Hallucinations - LLMs are good at generating good looking text, however it may not be factual.
RAG can help LLMs create text based on text that is factual. 
2. Many LLMs are trained on internet data, as such they have a good understanding of language. RAG allows us to use custom data. We can use customer support Q&A for chatting. We can retrieve relevant snippets of text for example. We can retrieve the snippets and then use an LLM to craft an answer from these snippets. 
3. Why run it locally. We do not have to wait for any transfers. Cost is another big factor. If we own our own hardware, we can save on large amounts of costs. Furthermore, there is no vendor locking, when we run our own software, hardware. If OpenAI or another large internet company shuts down, we can still run the buisness. Privacy - Id you have documentation, maybe you do not want to send it to an API. You want to setup an LLM and run it on your own hardware.

## What are we going to build?
https://whimsical.com/simple-local-rag-workflow-39kToR3yNf7E8kY4sS2tjV

1. Open a pdf document.
2. Format the text of the PDF textbook ready for an embedding model.
3. Embed all the chunk of text in the textbook and turn them into numerical representations (embedding) which can store for later. 
4. Build retrieval system that uses vector search to find the relevant chunks of text based on query. 
5. Create a prompt that incorperates the retrieved prieces of text. 
6. Generate the answer to a query based on the passages based on the passages of the textbook with an LLM.


## 1. Document pre-processing and embedding creation 

Ingridients: PDF document of choice (could be any kind of document.) and an embedding model of choice. 
1. Import PDF document
2. Process text for embedding (splitting into chunks of sentences)
3. Embedd textchunks with embedding model.
4. Save embedding to file for later (embeddings will store on file for many years until you loose them on hd).

In [20]:
# Programatically get the pdf document 
import os 
import requests 

# Get PDF document:
pdf_path = "./data/human-nutrition-text.pdf"

# Download the PDF:
if not os.path.exists(pdf_path):
    print("[INFO] File does not exist, downloading....")

    # Enter the URL of the PDF: 
    url = "https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"

    # Local Filename to save the file:
    filename = pdf_path

    # Send a GET request:
    response = requests.get(url=url)

    # Check if the request was successfull:
    if response.status_code == 200:
        # Open file and save it (wb = write binary)
        with open(filename, "wb") as file:
            file.write(response.content)
        print(f"[INFO] The file has been downloaded and saved as {filename}")
    else:
        print(f"[INFO] Failed to download the file. Status code: {response.status_code}")
else:
    print(f"[INFO] The file already exists")


[INFO] The file already exists


We got a PDF as such we can open it. We can use PyMUPDF which seems to be the best for PDF reading with the best Text formatting.

In [24]:
import pdfplumber
from tqdm.auto import tqdm

def text_formatter(text: str) -> str:
    """Performs basic formatting on text."""
    # Replace newlines and tabs with spaces
    text = text.replace('\n', ' ').replace('\t', ' ')
    
    # Strip leading/trailing whitespace
    text = text.strip()
    
    return text

def open_and_read_pdf(pdf_path: str) -> list[dict]:
    """
    Opens a PDF file, reads its text content page by page, and collects statistics.
    """
    reader = pdfplumber.open(pdf_path)
    pages_and_texts = []
    for page_number, page in tqdm(enumerate(reader.pages)):
        text = page.extract_text()
        text = text_formatter(text)
        
        pages_and_texts.append({
            "page_number": page_number,
            "page_char_count": len(text),
            "page_word_count": len(text.split()),
            "page_sentence_count_raw": text.count('.') + text.count('!') + text.count('?'),
            "page_token_count": len(text) // 4,
            "text": text
        })
    return pages_and_texts

pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[6:10]

0it [00:00, ?it/s]

[{'page_number': 6,
  'page_char_count': 998,
  'page_word_count': 152,
  'page_sentence_count_raw': 0,
  'page_token_count': 249,
  'text': 'The Cardiovascular System 82 University of Hawai‘i at Mānoa Food Science and Human Nutrition Program and Human Nutrition Program Central Nervous System 94 University of Hawai‘i at Mānoa Food Science and Human Nutrition Program and Human Nutrition Program The Respiratory System 99 University of Hawai‘i at Mānoa Food Science and Human Nutrition Program and Human Nutrition Program The Endocrine System 106 University of Hawai‘i at Mānoa Food Science and Human Nutrition Program and Human Nutrition Program The Urinary System 110 University of Hawai‘i at Mānoa Food Science and Human Nutrition Program and Human Nutrition Program The Muscular System 117 University of Hawai‘i at Mānoa Food Science and Human Nutrition Program and Human Nutrition Program The Skeletal System 120 University of Hawai‘i at Mānoa Food Science and Human Nutrition Program and Human